In [56]:
import numpy as np
import os
import h5py
import glob
from pathlib import Path
import cv2
import z5py

In [38]:
frame_numbers = [] 

for file in  glob.glob("E:/all_video_frames_from_avi/*.png") :
	frame_number = file.split("\\")[-1].split(".")[0].split("frame")[-1] 
	frame_numbers.append(frame_number)
 
 
frame_numbers_sorted = np.sort(np.array(frame_numbers, dtype=int))

In [ ]:


data_set_name = "images"
h5file = 'small_moto.h5'
with h5py.File(h5file,'w') as  h5f:

	#just for dims 
	base_file_name = "E:/all_video_frames_from_avi\\frame"
	tmp_img_name = base_file_name + str(frame_numbers_sorted[0]) + ".png"
	img = cv2.imread(tmp_img_name, cv2.IMREAD_GRAYSCALE)
	IMG_WIDTH = img.shape[1] #int(img.shape[1]/10)
	IMG_HEIGHT = img.shape[0] #int(img.shape[0]/10)
    
    #make stack
	img_ds = h5f.create_dataset(data_set_name,
        shape=(len(frame_numbers_sorted[:50]), IMG_HEIGHT, IMG_WIDTH), dtype=int)

	#loopa
	for idx, frame_number in enumerate(frame_numbers_sorted[:50]):
		tmp_img_name = base_file_name + str(frame_number) + ".png"
		img = cv2.imread(tmp_img_name, cv2.IMREAD_GRAYSCALE)
		img_resize = cv2.resize( img, (IMG_WIDTH, IMG_HEIGHT) )
		img_ds[idx] = img_resize.astype('int32')


In [63]:
h5_file_path = 'small_moto.h5'
h5_dataset_name = 'images'

n5_output_path = 'output_n5_directory.n5'
n5_dataset_name = 'converted_data'

# chunk_size will be dynamically set to the full data shape to create a single chunk
# chunk_size = (64, 64, 64) # This line is no longer needed as it will be dynamic

compression_type = 'gzip'

try:
    with h5py.File(h5_file_path, 'r') as h5_f:
        if h5_dataset_name not in h5_f:
            raise ValueError(f"Dataset '{h5_dataset_name}' not found in '{h5_file_path}'.")
        data = h5_f[h5_dataset_name][()]

    # Set chunk_size to the full shape of the data to store it as a single chunk
    chunk_size = data.shape

    n5_f = z5py.File(n5_output_path, use_zarr_format=False)

    n5_dataset = n5_f.create_dataset(
        n5_dataset_name,
        shape=data.shape,
        chunks=chunk_size, # Now uses the full data shape
        dtype=data.dtype,
        compression=compression_type
    )
    n5_dataset[:] = data

    print(f"Conversion successful: '{h5_file_path}' -> '{n5_output_path}/{n5_dataset_name}' (stored as a single chunk)")

except Exception as e:
    print(f"An error occurred during conversion: {e}")


Conversion successful: 'small_moto.h5' -> 'output_n5_directory.n5/converted_data' (stored as a single chunk)
